In [1]:
import sys
sys.path.append("..")

In [2]:
import pandas as pd
import numpy as np
from sklearn.datasets import make_blobs
from planetoids import planetoids as pt

X, y = make_blobs(n_samples=1000, n_features=2, random_state=42)
data = pd.DataFrame(X)
data['Cluster'] = y

C:\ProgramData\Anaconda3\lib\site-packages\tqdm\autonotebook\__init__.py:18: TqdmExperimentalWarning:

Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)



In [74]:
data = pd.read_csv("test_data.csv")
planet = pt.Planetoid(data, "0", "1", "Cluster", random_state=42)

In [105]:
planet.contours[0][0]

[array([[ 7.58409848e+01, -1.41765516e+01],
        [ 7.58409421e+01, -1.41763369e+01],
        [ 7.56862596e+01, -1.31692159e+01],
        [ 7.56862343e+01, -1.31689701e+01],
        [ 7.56862334e+01, -1.31687231e+01],
        [ 7.57480122e+01, -1.18258479e+01],
        [ 7.57480830e+01, -1.18252089e+01],
        [ 7.61631297e+01, -9.47459437e+00],
        [ 7.61631619e+01, -9.47440126e+00],
        [ 7.64783066e+01, -7.46004174e+00],
        [ 7.64784261e+01, -7.45861161e+00],
        [ 7.64978243e+01, -5.78006830e+00],
        [ 7.64978249e+01, -5.77998199e+00],
        [ 7.65037185e+01, -3.76510620e+00],
        [ 7.65037315e+01, -3.76485940e+00],
        [ 7.65037688e+01, -3.76461510e+00],
        [ 7.67093137e+01, -2.75751296e+00],
        [ 7.67093736e+01, -2.75727813e+00],
        [ 7.67094561e+01, -2.75705023e+00],
        [ 7.71303712e+01, -1.74991746e+00],
        [ 7.71304764e+01, -1.74969685e+00],
        [ 7.71306027e+01, -1.74948761e+00],
        [ 7.78079833e+01, -7.422

In [102]:
contours = None
with open("contours.pickle", "rb") as handle:
    contours = pickle.load(handle)
contours

{}

In [3]:
import pickle

In [4]:
planet = pt.Planetoid(data, 0, 1, 'Cluster', random_state=42)

In [5]:
# cset = ax.contour(xx, yy, f, colors="k", levels=topography_levels)

In [53]:
contours = None
with open("contours.pickle", "rb") as handle:
    contours = pickle.load(handle)

checks = []
for k, v in planet.contours.items():
    for ix, vv in enumerate(v):
        for ixx, vvv in enumerate(vv):
            checks.append(np.all(vvv == contours[k][ix][ixx]))
assert np.all(checks) == True

In [65]:
test_get_all_contours(planet)

Testing get all contours
True
True


In [64]:
def test_rescale_coordinates(planet):
    #np.save('rescaled', planet.data[['Latitude', 'Longitude']])
    planet._rescale_coordinates()
    return np.array_equal(planet.data[['Latitude', 'Longitude']].values, np.load('rescaled.npy'))
    

def test_get_all_contours(planet):
    print("Testing get all contours")
    # with open('contours.pickle', 'wb') as handle:
    #     pickle.dump(planet.contours, handle)

    #planet.get_all_contours(topography_levels=20, lighting_levels=20, relief_density=3)

    contours = None
    with open("contours.pickle", "rb") as handle:
        contours = pickle.load(handle)

    checks = []
    for k, v in planet.contours.items():
        for ix, vv in enumerate(v):
            for ixx, vvv in enumerate(vv):
                checks.append(np.all(vvv.round(6) == contours[k][ix][ixx].round(6)))
    print(np.all(checks))
    print(np.all(checks) == True)
    
def test_get_contours(planet):
    cntrs = planet._get_contours(0,
                         planet.data[planet.data['Cluster'] == 0],
                         topography_levels=20, lighting_levels=20, relief_density=3)
    contours = None
    with open('cntrs_0.pickle', 'rb') as handle:
        contours = pickle.load(handle)
    if np.testing.assert_equal(cntrs, contours, verbose=True) == None:
        return True
    else:
        return False
    
def test_get_contour_verts(planet, subset):
    import scipy.stats as st
    import matplotlib.pyplot as plt
    
    fig = plt.figure(figsize=(8, 8))
    ax = fig.gca()
    
    y = subset["Latitude"].values
    x = subset["Longitude"].values

    # Define the borders
    deltaX = (max(x) - min(x)) / 3
    deltaY = (max(y) - min(y)) / 3
    xmin = max(-180, min(x) - deltaX)
    xmax = min(180, max(x) + deltaX)
    ymin = max(-90, min(y) - deltaY)
    ymax = min(90, max(y) + deltaY)
    
    xx, yy = np.mgrid[
    xmin : xmax : (30 * 10 + 1j),
    ymin : ymax : (30 * 10 + 1j),
    ]
    
    positions = np.vstack([xx.ravel(), yy.ravel()])
    values = np.vstack([x, y])
    kernel = st.gaussian_kde(values)
    f = np.reshape(kernel(positions).T, xx.shape)
    cset = ax.contour(xx, yy, f, colors="k", levels=20)
    plt.close(fig)
    verts = None
    with open('verts_0.pickle', 'rb') as handle:
        verts = pickle.load(handle)
    if np.testing.assert_equal(verts, planet._get_contour_verts(cset), verbose=True) == None:
        return True
    else:
        return False
    
def test_clean_contours(planet):
    global f
    verts = None
    clean_verts = None
    
    with open('verts_0.pickle', 'rb') as handle:
        verts = pickle.load(handle)
        
    with open('clean_verts_0.pickle', 'rb') as handle:
        clean_verts = pickle.load(handle)
        
    if np.testing.assert_equal(clean_verts, planet._clean_contours(verts), verbose=True) == None:
        return True
    else:
        return False
    
def test_calculate_hillshade(planet, subset):
    import scipy.stats as st
    import matplotlib.pyplot as plt

    y = subset["Latitude"].values
    x = subset["Longitude"].values

    # Define the borders
    deltaX = (max(x) - min(x)) / 3
    deltaY = (max(y) - min(y)) / 3
    xmin = max(-180, min(x) - deltaX)
    xmax = min(180, max(x) + deltaX)
    ymin = max(-90, min(y) - deltaY)
    ymax = min(90, max(y) + deltaY)
    
    xx, yy = np.mgrid[
    xmin : xmax : (30 * 10 + 1j),
    ymin : ymax : (30 * 10 + 1j),
    ]
    
    positions = np.vstack([xx.ravel(), yy.ravel()])
    values = np.vstack([x, y])
    kernel = st.gaussian_kde(values)
    f = np.reshape(kernel(positions).T, xx.shape)
    
    return np.array_equal(planet._calculate_hillshade(np.rot90(f), 315, 45), np.load('hillshade.npy'))
    

In [10]:
test_rescale_coordinates(planet)

test_get_all_contours(planet)

test_get_contours(planet)

test_get_contour_verts(planet, planet.data[planet.data['Cluster'] == 0])

test_clean_contours(planet)

test_calculate_hillshade(planet, planet.data[planet.data['Cluster'] == 0])

True

In [8]:
# np.save('hillshade', hs)

In [91]:
with open('contours.pickle', 'wb') as handle:
    pickle.dump(planet.contours, handle)